### Import some useful libraries

In [4]:
import requests
import json
import getpass, os
import ast
import uuid

In [5]:
def get_Token(username, password):
    '''Function to get Token for username provided'''
    try:
        url = "https://xray-backend.azurewebsites.net/api-token-auth/"
        r = requests.post(url, data = {"username": username, "password": password})

        if r.status_code != 200:
            print("###################################################")
            print("###################################################")
            print("Provide a valid username and password.....")
            print("###################################################")
            print("###################################################")
        else:

            print("###################################################")
            print("###################################################")
            print("Authorized.....")
            print("###################################################")
            print("###################################################")

        token = json.loads(r.text)['token']
    
    except Exception as e:
        
        print("Error in logging in:: " + str(e))
        raise Exception(e)

    return token

### Login with Datascientist Id and password

In [6]:
token = ""
while len(token) == 0:
    
    user = input("Enter UserID\n")
    password = getpass.getpass("Enter password...\n")
    token = get_Token(user,password)


Enter UserID
datascientist
Enter password...
········
###################################################
###################################################
Authorized.....
###################################################
###################################################


### Enter Parameters

In [10]:
print("Provide parameters to train your model..")
print("###################################################")
print("###################################################")

parameters = {'message':'training',
              "datasetId":0,"modelName":'model',"optimizer":'adam',
              "lossFunction":'binary_crossentropy',"learningRate":0.001,
              "epochs":1,"stepsPerEpoch":'None',"initialEpoch":0,"cycles":1,
              "validationSteps":'None',"batchSize":16,"featurewise_center":False,
              "samplewise_center":False,"featurewise_std_normalization":False,
              "samplewise_std_normalization":False,"zca_whitening":False,
              "rotation_range":0,"width_shift_range":0.0,
              "height_shift_range":0.0,"brightness_range":'None',
              "shear_range":0.0,"zoom_range":0.0,"channel_shift_range":0.0,
              "fill_mode":"nearest","cval":0.0,"horizontal_flip":False,
              "vertical_flip":False,"rescale":'None',"data_format":'None',
              "validation_split":0.1,"dtype":'None',"shuffle":True}

#################################
for i in parameters.keys():
    inp = ''
    if i != "message":
        
        inp = input(f"Enter {i}\n default: {parameters[i]}\t")
        
    if inp:
        parameters[i] = inp
###############################
#get values for metrics and layers to be freezed from user

excluded = {"layers_non_trainable":[],"metrics":[]}   


for i in excluded.keys():
    
    inp = input(f"Enter values for {i} separated by comma")
    for j in inp.split(','):
        excluded[i].append(j)

parameters.update(excluded)
####################################

#Insert all parameters in Experiment table   
parameters['metrics'] = str(parameters['metrics'])
#######################################################
url = "http://127.0.0.1:8000/experiments/"
# url = 'https://xray-backend.azurewebsites.net/experiments/'
header = {'Authorization' : f"Token {token}"}
re = requests.post(url,headers= header,data=parameters)
print(re.status_code)
print(re.content)

#######################################################

Provide parameters to train your model..
###################################################
###################################################
Enter datasetId
 default: 0	80
Enter modelName
 default: model	vggnet
Enter optimizer
 default: adam	
Enter lossFunction
 default: binary_crossentropy	
Enter learningRate
 default: 0.001	
Enter epochs
 default: 1	10
Enter stepsPerEpoch
 default: None	
Enter initialEpoch
 default: 0	
Enter cycles
 default: 1	2
Enter validationSteps
 default: None	
Enter batchSize
 default: 16	
Enter featurewise_center
 default: False	
Enter samplewise_center
 default: False	
Enter featurewise_std_normalization
 default: False	
Enter samplewise_std_normalization
 default: False	
Enter zca_whitening
 default: False	
Enter rotation_range
 default: 0	
Enter width_shift_range
 default: 0.0	
Enter height_shift_range
 default: 0.0	
Enter brightness_range
 default: None	
Enter shear_range
 default: 0.0	
Enter zoom_range
 default: 0.0	
Enter channel_shift_range
 default

### Upload model file and weights to the server before training

In [14]:
body_unicode = re.content.decode('utf-8')
content = json.loads(body_unicode)
print("Make sure model file and weights are in current directory !")
print("###################################################")
print("###################################################")
model_name = input("Enter your model file name here eg: vggnet....\n")
print("###################################################")
print("###################################################")

#Upload/Send model file to the server

# url_model = 'http://127.0.0.1:8000/upload/'
url_model = 'https://xray-backend.azurewebsites.net/upload/'
header = {'Authorization' : f"Token {token}"}
files = {'upload_file': open(f'{model_name}.py','rb')}
values = {'type': "model","experimentId": content['id']}
r = requests.post(url_model, headers = header, files=files, data=values)
if r.status_code == 200:
    print("###################################################")
    print("###################################################")
    print("Model file uploaded.....")
else:
    print("###################################################")
    print("###################################################")
    print("Error uploading....")

#Upload/Send model weights to the server
# url_weights = 'http://127.0.0.1:8000/upload/'
url_weights = 'https://xray-backend.azurewebsites.net/upload/'
header = {'Authorization' : f"Token {token}"}
files = {'upload_file': open(f'{model_name}_weights.pkl','rb')}
values = {'type': "weights","experimentId": content['id']}
r = requests.post(url_weights, headers = header, files=files, data=values)
if r.status_code == 200:
    print("###################################################")
    print("###################################################")
    print("Weights uploaded....")
else:
    print("###################################################")
    print("###################################################")
    print("Error uploading....")

Make sure model file and weights are in current directory !
###################################################
###################################################
Enter your model file name here eg: vggnet....
vggnet
###################################################
###################################################
###################################################
###################################################
Model file uploaded.....
###################################################
###################################################
Weights uploaded....


In [ ]:
# run this cell for callbacks or leave the cell unrexecuted

''' provide callbacks in the list below like: 
    my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=2),
    tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]'''

# my_callbacks = ['tf.keras.callbacks.EarlyStopping(patience=2)']

In [ ]:
#Dont run this cell incase my_callbacks is empty
# parameters['callbacks'] = my_callbacks

In [ ]:
# parameters

### Send training plan to the server

In [12]:
data = {"experiment_id":content['id']}
#Send training request to server
# url = "http://127.0.0.1:8000/training/"
url = "https://xray-backend.azurewebsites.net/training/"
header = {'Authorization' : f"Token {token}"}
r = requests.post(url, headers = header, data = data )
body_unicode = r.content.decode('utf-8')
content = json.loads(body_unicode)
print(content['message'])

Training request sent....
Updated weights will be available to download once training completed
